In [1]:
SRC = 'https://www.kyoukaikenpo.or.jp/~/media/Files/shared/hokenryouritu/r3/ippan/r30213tokyo.pdf'
DST = '2021-04.csv'

In [2]:
import datetime
import hashlib
import io
import pathlib
import urllib.request

try:
    import tabula
except ModuleNotFoundError:
    !pip install tabula-py
    import tabula

datetime.datetime.now().astimezone(datetime.timezone.utc).isoformat()

'2022-04-05T13:50:17.236517+00:00'

In [3]:
data = io.BytesIO()

with urllib.request.urlopen(SRC) as f:
    data.write(f.read())

hashlib.sha256(data.getvalue()).hexdigest()

'e53e2b9106efc93385d1ddd3567b394b21a812c0395317edd9eac103d0e74d20'

In [4]:
data.seek(0)

df = tabula.read_pdf(data, lattice=True, pages=1)[0]

df = df.iloc[:, [1, 2, 3, 9, 10]]

df.columns = """
標準報酬:等級
標準報酬:月額
報酬月額
全額
折半額
""".strip().split()

df = df[~df['標準報酬:等級'].isnull()]
df = df[df['標準報酬:等級'].str.contains(r'\([0-9]+\)')]
df['標準報酬:等級'] = df['標準報酬:等級'].str.replace(
    r'[^(]*\(([0-9]+)\)',
    lambda m: m.group(1),
    regex=True
)
df['標準報酬:等級'] = df['標準報酬:等級'].astype(int)
df = df.set_index('標準報酬:等級').sort_index()

assert df.shape[0] == 32, f'等級数 ({df.shape[0]}) に過不足があります。'

s = df['報酬月額'].str.replace(r'[^0-9~]', '', regex=True)
df_ = s.str.split('~', n=1, expand=True)
df_ = df_.rename(columns={0: '報酬月額:以上', 1: '報酬月額:未満'})
df = df.merge(df_, left_index=True, right_index=True)

del df['報酬月額']

for x in df.columns:
    df[x] = df[x].str.replace(',', '')

df['標準報酬:月額'] = df['標準報酬:月額'].astype(int)

for x in """
報酬月額:以上
報酬月額:未満
全額
折半額
""".strip().split():
    df[x] = df[x].astype(float)

df.loc[df.index.min(), '報酬月額:以上'] = float('-inf')
df.loc[df.index.max(), '報酬月額:未満'] = float('inf')

for x in """
折半額
""".strip().split():
    df['控除額'] = (df['折半額'] + .49).astype(int)

df = df["""
標準報酬:月額
報酬月額:以上
報酬月額:未満
全額
折半額
控除額
""".strip().split()]

df.to_csv(DST)

hashlib.sha256(pathlib.Path(DST).read_bytes()).hexdigest()

Got stderr: 4月 05, 2022 10:50:17 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored
4月 05, 2022 10:50:17 午後 org.apache.fontbox.ttf.CmapSubtable processSubtype14
警告: Format 14 cmap table is not supported and will be ignored



'e003abf2631bf4f7bbefb16eb73c96dfb8c875b78c5cee325ebb44513cb24fc6'